# Boilerplate and DB connection


In [ ]:
from datetime import datetime
from decimal import Decimal
import uuid, random

from sqlalchemy import (
    create_engine, Column, Integer, String, DateTime, Boolean,
    ForeignKey, Numeric, MetaData, text
)
from sqlalchemy.exc import OperationalError
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.dialects.postgresql import insert as pg_insert
DB_URL = (
    'postgresql+psycopg2://student:HSBCN323TestDb36111'
    '@database-1.cffitrsftriq.eu-central-1.rds.amazonaws.com:5432/postgres'
)
SCHEMA = 'hashguess'
engine = create_engine(DB_URL, isolation_level="SERIALIZABLE")
metadata = MetaData(schema=SCHEMA)
Base = declarative_base(metadata=metadata)
Session = scoped_session(sessionmaker(bind=engine))



<ipython-input-95-45ccb8f6316f>:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base(metadata=metadata)


# Tables

In [ ]:
class Player(Base):
    __tablename__ = 'player'
    player_id = Column(Integer, primary_key=True)
    username  = Column(String, unique=True, nullable=False)
    email     = Column(String, unique=True, nullable=False)
    balance   = Column(Numeric(12,2), nullable=False, default=Decimal('0.00'))

class Block(Base):
    __tablename__ = 'block'
    block_id     = Column(Integer, primary_key=True)
    block_hash   = Column(String, unique=True, nullable=True)   # null until generated
    actual_char  = Column(String(1), nullable=True)             # null until generated
    created_at   = Column(DateTime, default=datetime.utcnow, nullable=False)

class Bet(Base):
    __tablename__ = 'bet'
    bet_id       = Column(Integer, primary_key=True)
    player_id    = Column(Integer, ForeignKey(f"{SCHEMA}.player.player_id"), nullable=False)
    block_id     = Column(Integer, ForeignKey(f"{SCHEMA}.block.block_id"), nullable=False)
    prediction   = Column(String(1), nullable=False)
    stake        = Column(Numeric(12,2), nullable=False)
    placed_at    = Column(DateTime, default=datetime.utcnow, nullable=False)
    resolved     = Column(Boolean, default=False, nullable=False)
    is_win       = Column(Boolean, nullable=True)

# Schema

In [ ]:
def reset_schema():
    with engine.begin() as conn:
        conn.execute(text(f"DROP SCHEMA IF EXISTS {SCHEMA} CASCADE"))
        conn.execute(text(f"CREATE SCHEMA {SCHEMA}"))
    Base.metadata.create_all(engine)
    with engine.begin() as conn:
        conn.execute(text(f"""
        CREATE FUNCTION {SCHEMA}.check_active() RETURNS trigger LANGUAGE plpgsql AS $$
        DECLARE cnt INT;
        BEGIN
          SELECT COUNT(*) INTO cnt
            FROM {SCHEMA}.bet
           WHERE player_id = NEW.player_id
             AND resolved = FALSE;
          IF cnt >= 5 THEN
            RAISE EXCEPTION 'Too many active bets for player %', NEW.player_id;
          END IF;
          RETURN NEW;
        END;
        $$;
        CREATE TRIGGER trg_check_active
          BEFORE INSERT ON {SCHEMA}.bet
          FOR EACH ROW EXECUTE FUNCTION {SCHEMA}.check_active();
        """))


# Business Logic

In [ ]:
# 1) Create user
def create_user(username: str, email: str) -> int:
    s = Session()
    try:
        stmt = pg_insert(Player).values(
            username=username, email=email, balance=Decimal('0.00')
        ).on_conflict_do_nothing(index_elements=['username'])
        s.execute(stmt); s.commit()
        return s.query(Player).filter_by(username=username).one().player_id
    finally:
        Session.remove()

# 2) Increase balance by random [50,1000]
def increase_balance(player_id: int) -> Decimal:
    amt = Decimal(random.uniform(50, 1000)).quantize(Decimal('0.01'))
    s = Session()
    try:
        with s.begin():
            s.execute(text(
                f"UPDATE {SCHEMA}.player SET balance = balance + :amt WHERE player_id = :pid"
            ), {'amt': amt, 'pid': player_id})
        return amt
    finally:
        Session.remove()

# 3) Generate upcoming block (placeholder)
def generate_block() -> int:
    s = Session()
    try:
        with s.begin():
            last = s.query(Block).order_by(Block.block_id.desc()).first()
            if last and last.actual_char is None:
                raise Exception("Previous block still unresolved")
            res = s.execute(
                pg_insert(Block)
                .values(block_hash=None, actual_char=None)
                .returning(Block.block_id)
            )
            return res.scalar_one()
    finally:
        Session.remove()

# 4) Place a bet on that block
def make_bet(player_id: int, block_id: int, prediction: str, stake: Decimal) -> int:
    s = Session()
    try:
        with s.begin():
            # lock & check balance
            row = s.execute(text(
                f"SELECT balance FROM {SCHEMA}.player WHERE player_id=:pid FOR UPDATE"
            ), {'pid': player_id}).first()
            if not row or row.balance < stake:
                raise Exception("Insufficient balance")
            s.execute(text(
                f"UPDATE {SCHEMA}.player SET balance = balance - :stk WHERE player_id = :pid"
            ), {'stk': stake, 'pid': player_id})

            # verify block is upcoming
            blk = s.query(Block).get(block_id)
            if not blk or blk.actual_char is not None:
                raise Exception("Block not open for betting")

            b = Bet(
                player_id=player_id, block_id=block_id,
                prediction=prediction, stake=stake
            )
            s.add(b)
        return b.bet_id
    finally:
        Session.remove()

# 5a) Generate real hash for a given block
def generate_block_hash(block_id: int) -> str:
    h = uuid.uuid4().hex
    actual = h[-1]
    s = Session()
    try:
        with s.begin():
            s.execute(text(
                f"UPDATE {SCHEMA}.block SET block_hash=:h, actual_char=:c WHERE block_id=:bid"
            ), {'h': h, 'c': actual, 'bid': block_id})
        return h
    finally:
        Session.remove()

# 5b) Resolve **all** bets on any blocks that now have a hash
def resolve_all_bets() -> int:
    s = Session()
    try:
        with s.begin():
            res = s.execute(text(f"""
                UPDATE {SCHEMA}.bet AS bt
                   SET resolved = TRUE,
                       is_win = (bt.prediction = b.actual_char)
                  FROM {SCHEMA}.block AS b
                 WHERE bt.block_id = b.block_id
                   AND b.actual_char IS NOT NULL
                   AND bt.resolved = FALSE
            """))
            return res.rowcount
    finally:
        Session.remove()

# Schema Reset

In [ ]:
reset_schema()

# Check create_user() perfomance.

In [ ]:
import time
from decimal import Decimal

def perf_test_user_setup(n: int = 10000):
    """
    Measures throughput of the full user‐creation setup:
      1) create_user(username, email) → balance = 0.00
      2) increase_balance(player_id)   → random credit 50.00–1000.00
    for n distinct users perf_user_0 … perf_user_{n-1}.
    """
    start = time.time()
    successes = 0

    for i in range(n):
        username = f'perf_user_{i}'
        email    = f'{username}@example.com'
        try:
            # 1) create user (balance initialized to 0.00)
            uid = create_user(username, email)
            # 2) credit a random amount
            credited = increase_balance(uid)
            successes += 1
        except Exception as e:
            print(f"[{i}] Failed setup for {username}: {e}")

    elapsed = time.time() - start
    print(f"Completed setup for {successes}/{n} users in {elapsed:.2f}s → {successes/elapsed:.0f} ops/s")

In [ ]:
perf_test_user_setup(10)

Completed setup for 10/10 users in 10.07s → 1 ops/s


# Check betting perfomance

In [ ]:
import random
import time
from decimal import Decimal

def perf_test_betting_with_users(n_bets: int = 1000, stake: Decimal = Decimal('1.00')):
    """
    Tests betting throughput using multiple existing users created beforehand.
    Assumes perf_test_user_setup() was already run.
    """

    # Fetch all users with sufficient balance
    s = Session()
    users = s.query(Player.player_id).filter(Player.balance >= stake).all()
    s.close()

    user_ids = [u.player_id for u in users]
    if not user_ids:
        print("No users with sufficient balance found. Run perf_test_user_setup() first.")
        return

    print(f"Using {len(user_ids)} users for betting test.")

    bets_made = 0
    start = time.time()

    while bets_made < n_bets:
        # Ensure all previous blocks are resolved before creating new block
        resolve_all_bets()
        block_id = generate_block()

        for _ in range(min(100, n_bets - bets_made)):  # batch 100 bets per block
            uid = random.choice(user_ids)
            guess = random.choice('0123456789abcdef')
            try:
                make_bet(uid, block_id, guess, stake)
                bets_made += 1
            except Exception as e:
                print(f"Bet skipped for user {uid}: {e}")

        # After placing bets, resolve them (except for the most recent block)
        generate_block_hash(block_id)
        # resolve_all_bets() leaves last block unresolved as expected

    elapsed = time.time() - start
    print(f"Placed {bets_made} bets in {elapsed:.2f}s → {bets_made/elapsed:.0f} ops/s")

In [ ]:
perf_test_betting_with_users(10)
perf_test_betting_with_users(10)

Using 10 users for betting test.


<ipython-input-44-311ac7fd9bf5>:59: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  blk = s.query(Block).get(block_id)


Placed 10 bets in 11.25s → 1 ops/s
Using 10 users for betting test.
Placed 10 bets in 11.17s → 1 ops/s


# Isolation Test

In [110]:
import pytest
from threading import Thread
from sqlalchemy.exc import OperationalError, IntegrityError
from decimal import Decimal

def test_concurrent_bets_do_not_exceed_max_active():
    """
    Tests that concurrent bet placements do not exceed the maximum allowed active bets (5)
    for a player, ensuring database trigger and isolation level enforce the constraint.
    """
    reset_schema()

    # Setup user with sufficient balance
    username = "test_user_concurrent"
    email = f"{username}@example.com"
    user_id = create_user(username, email)
    increase_balance(user_id)  # Add random credit (>=50)

    # Generate a block to bet on
    block_id = generate_block()

    # Pre-place 4 bets to reach near the limit

    for _ in range(4):
        guess = random.choice('0123456789abcdef')
        make_bet(user_id, block_id,guess, Decimal('10.00'))

    # Track exceptions from threads
    exceptions = []
    def get_active_bets_count(player_id):
      s = Session()
      count = s.query(Bet).filter(Bet.player_id == player_id, Bet.resolved == False).count()
      s.close()
      return count
    def attempt_bet():
        """Thread function to place one additional bet."""
        try:
            bet_value = Decimal(random.choice('123456'))
            print(f"Attempting bet for user {user_id}, current active bets: {get_active_bets_count(user_id)}, with value{bet_value}")
            make_bet(user_id, block_id, '?',bet_value)
            print(f"Bet placed successfully for user {user_id}")
        except Exception as e:
            exceptions.append(e)

    # Spawn two threads to attempt placing the 5th and 6th bets concurrently
    t1 = Thread(target=attempt_bet)
    t2 = Thread(target=attempt_bet)
    t1.start()
    t2.start() # Fixed: Removed extra closing parenthesis
    time.sleep(1)
    t1.join()
    t2.join()
    print(f"Active bets after 6th bet attempt: {get_active_bets_count(user_id)}")
    # Verify results
    s = Session()
    try:
        active_bets = s.query(Bet).filter(
            Bet.player_id == user_id,
            Bet.resolved == False
        ).count() # Fixed: Corrected indentation
        assert active_bets <= 5, f"Active bets exceeded limit (found {active_bets})"

        # Ensure at least one failure occurred (trigger or serialization error)
        assert len(exceptions) >= 1, "Expected concurrency-related exception"
    finally:
        s.close()
        Session.remove()

In [111]:
test_concurrent_bets_do_not_exceed_max_active()

<ipython-input-63-311ac7fd9bf5>:59: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  blk = s.query(Block).get(block_id)


Attempting bet for user 1, current active bets: 4, with value6
Attempting bet for user 1, current active bets: 4, with value5
Bet placed successfully for user 1
Active bets after 6th bet attempt: 5
